# Convert fore names to simplified (one word) fore names

In [1]:
import collections
import string
import pandas

import pubmedpy.names

In [2]:
name_df_paths = [
    'data/pubmed/authors.tsv.xz',
    'data/pmc/authors.tsv.xz',
    'data/iscb/keynotes.tsv',
]
path_to_df = {}
for path in name_df_paths:
    name_df = pandas.read_csv(path, keep_default_na=False, na_values=[''], sep='\t')
    path_to_df[path] = name_df

In [3]:
def simplify_names(dfs, name_column):
    names = pandas.concat(df[name_column] for df in dfs)
    simple_name_df = names.value_counts(dropna=True).reset_index()
    simple_name_df.columns = [name_column, 'n_authors']
    assert simple_name_df[name_column].isna().sum() == 0
    simplifier = getattr(pubmedpy.names, f'simplify_{name_column}')
    simple_name_df[f'{name_column}_simple'] = simple_name_df[name_column].map(lambda x: simplifier(x, lower=True))
    simple_name_df = simple_name_df.sort_values(name_column)
    return simple_name_df

In [4]:
simple_fore_name_df = simplify_names(path_to_df.values(), 'fore_name')
simple_fore_name_df

,fore_name,n_authors,fore_name_simple
36063,(Max) Zong-Ming,1,max
27945,", Yuxin",1,yuxin
24,A,373,None
2286,A A,17,None
17538,A Aldo,2,aldo
...,...,...,...
21724,Špela,2,špela
17058,Žiga,2,žiga
34616,Živa,1,živa
24255,Živadin,2,živadin


In [5]:
simple_last_name_df = simplify_names(path_to_df.values(), 'last_name')
simple_last_name_df

,last_name,n_authors,last_name_simple
36523,Rozowsky,1,rozowsky
36132,Slater,1,slater
35628,'t Hart,1,t hart
2599,'t Hoen,14,t hoen
38525,(Holly) Yang,1,holly) yang
...,...,...,...
27860,Žárský,2,žárský
39442,železný,1,železný
36464,žurauskienė,1,žurauskienė
26558,‘t Hoen,2,‘t hoen


In [6]:
simple_fore_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,fore_name,n_authors,fore_name_simple
102,Jörg,180,jörg
661,Ruben,47,ruben
1608,Magdalena,22,magdalena
2639,Lorraine,15,lorraine
2976,Milan,13,milan
6146,Gustavo A.,6,gustavo
7937,Josephine,5,josephine
7539,Louis M,5,louis
8751,Hung D,4,hung
9412,Ayman,4,ayman


In [7]:
simple_last_name_df.sample(n=20, random_state=0).sort_values('n_authors', ascending=False)

,last_name,n_authors,last_name_simple
102,Cohen,127,cohen
1372,Heckerman,21,heckerman
1567,Woo,19,woo
4462,Marín,9,marín
9065,Bähler,5,bähler
10438,Hemenway,4,hemenway
10872,Renou,4,renou
13069,Vervier,4,vervier
10733,Slawski,4,slawski
12171,Sakamoto,4,sakamoto


In [8]:
# Most common last names
(
    simple_last_name_df
    .groupby('last_name_simple')
    .n_authors.sum()
    .sort_values(ascending=False)
    .reset_index()
    .head(3)
)

,last_name_simple,n_authors
0,wang,3232
1,li,2716
2,zhang,2652


In [9]:
simple_fore_name_df.to_csv('data/names/fore-names.tsv.xz', sep='\t', index=False)
simple_last_name_df.to_csv('data/names/last-names.tsv.xz', sep='\t', index=False)